In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
import json
import time
import random
from ast import literal_eval
import os

C:\Users\iponc\AppData\Local\Temp\ipykernel_16092\4074257894.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def extraer_tabla(soup):
    tablas = soup.find_all('table')
    return tablas

In [3]:
def extraer_valores(texto):
    # Extraer número de patente
    patent = re.findall(r'<span class="highlight">EP<span class="highlight">(\d+)</span></span>', texto)
    if len(patent) == 0:
        patent = ''
    else:
        patent = 'EP' + patent[0]

    # Extraer nombre de la patente
    patent_name = re.findall(r'</span></span> - (.*?)</a>', texto)
    if len(patent_name) == 0:
        patent_name = ''
    else:
        patent_name = patent_name[0].strip()  

    # Extraer Status
    status = re.findall("""Status</td><td class="t2" colspan="3">.*?<br/>""", texto)
    if len(status) == 0:
        status = ''
    else:
        status = status[0]
        status = status.replace("""Status</td><td class="t2" colspan="3">""","").replace('<br/>','').strip()

    #Most Recent Event date
    most_recent_event_date = re.findall(r'Most recent event.*?<td class="t2">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event_date) == 0:
        most_recent_event_date = ''
    else:
        most_recent_event_date = most_recent_event_date[0].strip()

    #Most recent event
    most_recent_event = re.findall(r'Most recent event.*?<td class="t3">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event) == 0:
        most_recent_event = ''
    else:
        most_recent_event = most_recent_event[0].replace('<br/>', '').replace('\r\n', '').replace('\t', '').replace('\xa0', ' ').strip()

    #Divisional
    divisional_pattern = r'<td class="th"[^>]*>Divisional application\(s\)<\/td>(.*?)<\/tr>'
    divisional_match = re.search(divisional_pattern, texto, re.DOTALL)
    divisional_dict = {}
    if divisional_match:
        divisional_text = divisional_match.group(1)
        divisional_apps = re.findall(r'(EP\d+\.\d+)\s*\/\s*<a.*?>(EP\d+)<\/a>', divisional_text)
        divisional_dict = dict(divisional_apps)

    #Parent
    parent_pattern = r'<td class="th">Parent application\(s\).*?<td class="t2" colspan="3">(.*?)</td>'
    parent_match = re.search(parent_pattern, texto, re.DOTALL | re.IGNORECASE)
    parent_dict = {}
    if parent_match:
        parent_text = parent_match.group(1)
        parent_apps = re.findall(r'(EP\d+\.\d+)\s*\/\s*<a.*?>(EP\d+)</a>', parent_text)
        parent_dict = dict(parent_apps)


    # Extraer solicitantes
    pattern = re.compile(
        r'<td class="t2" colspan="3">(For all designated states)<br/>\r\n\t(.*?)(<br/>\r\n\t(.*?))?(<br/>\r\n\t(.*?))?(<br/>\r\n\t(.*?))?(<br/>\r\n\t(.*?))?\r\n\t  / (.*?)<br/>'
    )
    matches = pattern.findall(texto)

    applicants_dict = []
    for match in matches:
        address_parts = [match[1]]
        
        if match[3]:
            address_parts.append(match[3])
        if match[5]:
            address_parts.append(match[5])
        if match[7]:
            address_parts.append(match[7])
        if match[9]:
            address_parts.append(match[9])
            
        address = ', '.join(address_parts)
        country = match[10] if len(match) > 10 else ''
        
        applicant_info = {
            'for': match[0],
            'name': match[1],
            'address': address,
            'country': country
        }
        applicants_dict.append(applicant_info)

        
    # Extraer inventores
    inventors = re.findall(r'<td class="t2" colspan="3">(\d+)\xa0/\r\n\t(.*?)<br/>\r\n\t(.*?)<br/>\r\n\t(.*?)\r\n\t  / (.*?)<br/>', texto)
    inventors_dict = []
    for inv in inventors:
        inventor_info = {
            'number': inv[0],
            'name': inv[1],
            'address': inv[2] + ', ' + inv[3],
            'country': inv[4]
        }
        inventors_dict.append(inventor_info)

    # Extraer publicaciones 
    publications = re.findall(r'<td class="th">Type:\r\n\s*</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">No.:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Date:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Language:</td>\s*<td class="t2" colspan="2">(.*?)</td>', texto)
    publications_list = []
    for pub in publications:
        # Extraer el texto del enlace si existe, o usar el texto plano si no hay enlace
        type_text = re.search(r'>([^<]+)</a>$', pub[0])
        if type_text:
            pub_type = type_text.group(1).strip()
        else:
            pub_type = re.sub(r'\xa0', ' ', pub[0]).strip()
        publication_info = {
            'type': pub_type,
            'number': re.sub(r'<.*?>', '', pub[1]).strip(),
            'date': pub[2].strip(),
            'language': pub[3].strip()
        }
        publications_list.append(publication_info)

    # Extraer Priority numbers
    priority_section = re.search(r'<td class="th" rowspan="\d+">Priority number, date</td>(.*?)<td class="t2" colspan="3">\[', texto, re.DOTALL)
    if not priority_section:
        priority_numbers = []
    else:
        priority_data = priority_section.group(1)
        priority_numbers = re.findall(r'<td class="t2">([A-Z]+\d+)</td>', priority_data)


    # Extraer clasificaciones IPC
    ipc_pattern = r'IPC:\s*</td>\s*<td class="t2">\s*(.*?)\s*</td>'
    ipc_matches = re.findall(ipc_pattern, texto, re.DOTALL)
    ipc_list = [re.sub(r'\s+', ' ', ipc).strip() for ipc in ipc_matches]

    # Extraer clasificaciones CPC
    cpc_pattern = r'CPC:</td>\s*<td class="t2" colspan="2">(.*?)</td>'
    cpc_matches = re.findall(cpc_pattern, texto, re.DOTALL)
    
    cpc_list = []
    for match in cpc_matches:
        soup = BeautifulSoup(match, 'html.parser')
        b_elements = soup.find_all('b') # Encontrar todos los elementos <b> dentro del bloque
        
        # Extraer el texto y limpiarlo
        items = [elem.get_text().strip() for elem in b_elements]
        cpc_list.extend(items)


    # Opponent
    opposition_pattern = r'<td class="th">\s*Opponent\(s\)</td>\s*<td class="t2" colspan="2">(.*?)</td>'
    opposition_raw = re.findall(opposition_pattern, texto, re.DOTALL)

    oppositions_list = []
    if opposition_raw:
        opposition_entries = re.split(r'</td>\s*</tr><td class="th">&nbsp;</td><td class="t2" colspan="2">', opposition_raw[0])
        for entry in opposition_entries:
            if entry.strip():
                # Limpiar y dividir la entrada
                parts = entry.strip().split('<br/>')
                if len(parts) >= 7:
                    opposition_info = {
                        'number': parts[0].strip(),
                        'date_received': parts[1].strip(),
                        'date_decision': parts[2].strip(),
                        'status': parts[3].strip(),
                        'name': parts[4].strip().replace('&nbsp;', ' '),
                        'address': ' '.join(parts[5:7]).strip().replace('&nbsp;', ' '),
                        'representative': ' '.join(parts[7:9]).strip().replace('&nbsp;', ' ')
                    }
                    oppositions_list.append(opposition_info)

    # Crear DataFrame
    df = pd.DataFrame({
        'Patent': [patent],
        'Patent_Name': [patent_name],
        'Status': [status],
        'Most_Recent_Event_Date': [most_recent_event_date],
        'Most_Recent_Event': [most_recent_event],
        'Divisional_applications': [json.dumps(divisional_dict)],
        'Parent_applications': [json.dumps(parent_dict)],
        'Inventors': [json.dumps(inventors_dict)],
        'Publications': [json.dumps(publications_list)],
        'Priority numbers': [priority_numbers],
        'Applicants': [json.dumps(applicants_dict)],
        'IPC': [json.dumps(ipc_list)],
        'CPC': [json.dumps(cpc_list)],
        'Opponent': [json.dumps(oppositions_list)]
    })
    
    return df


In [5]:
carpeta = 'C:/Users/iponc/Downloads/HTMLS'  # Ruta de la carpeta HTMLS
df_general = pd.DataFrame()  

for nombre_archivo in os.listdir(carpeta):
    try:
        ruta_completa = os.path.join(carpeta, nombre_archivo)
        if os.path.isfile(ruta_completa):
            print(f"Procesando archivo: {nombre_archivo}")
            
            with open(ruta_completa, 'r', encoding='utf-8') as file:
                contenido = file.read()
                soup = BeautifulSoup(contenido, 'html.parser')
                mytable = extraer_tabla(soup)[0]
                df_actual = extraer_valores(str(mytable))
                
                df_general = pd.concat([df_general, df_actual], ignore_index=True)
                
    except Exception as e:
        print(f"Error inesperado al procesar el archivo {nombre_archivo}: {str(e)}")
        continue

print(f"Total de filas en el DataFrame general: {len(df_general)}")


Procesando archivo: EP0721594.txt
Procesando archivo: EP0737549.txt
Procesando archivo: EP0788617.txt
Procesando archivo: EP0885233.txt
Procesando archivo: EP1105513.txt
Procesando archivo: EP1127791.txt
Procesando archivo: EP1164138.txt
Procesando archivo: EP1167108.txt
Procesando archivo: EP1168032.txt
Procesando archivo: EP1250728.txt
Procesando archivo: EP1250983.txt
Procesando archivo: EP1250989.txt
Procesando archivo: EP1251150.txt
Procesando archivo: EP1443598.txt
Procesando archivo: EP1475410.txt
Procesando archivo: EP1489185.txt
Procesando archivo: EP1502461.txt
Procesando archivo: EP1613746.txt
Procesando archivo: EP1613796.txt
Procesando archivo: EP1614628.txt
Procesando archivo: EP1616678.txt
Procesando archivo: EP1663881.txt
Procesando archivo: EP1717123.txt
Procesando archivo: EP1730131.txt
Procesando archivo: EP1744959.txt
Procesando archivo: EP1761553.txt
Procesando archivo: EP1769893.txt
Procesando archivo: EP1775298.txt
Procesando archivo: EP1873124.txt
Procesando arc

In [6]:
df_general

,Patent,Patent_Name,Status,Most_Recent_Event_Date,Most_Recent_Event,Divisional_applications,Parent_applications,Inventors,Publications,Priority numbers,Applicants,IPC,CPC,Opponent
0,EP0721594,AUTONOMOUS CRUISE CONTROL,No opposition filed within time limit,28.12.2007,Lapse of the patent in a contracting state\nNe...,"{""EP01109826.6"": ""EP1167108""}",{},[],[],[US19930130585],[],"[""G01S13/60, B60K31/00, G01S13/93<br/>""]","[""B60K31/0008 (EP,KR,US);"", ""B60W30/146 (EP);""...",[]
1,EP0737549,Abrasive tape and process for producing it,No opposition filed within time limit,24.10.2003,No opposition filed within time limit,"{""EP02012459.0"": ""EP1250983""}",{},[],[],"[JP19950109088, JP19960005212]",[],"[""B24D3/00, B24D11/00, B24D3/28<br/>""]","[""B24B19/226 (EP,US);"", ""B24D3/00 (KR);"", ""B24...",[]
2,EP0788617,MINIATURE OPTICAL SCANNER FOR A TWO AXIS SCANN...,No opposition filed within time limit,04.01.2008,Lapse of the patent in a contracting state\nNe...,"{""EP01202983.1"": ""EP1168032""}",{},[],[],[US19940329508],[],"[""G02B26/08, G02B26/10, G02B7/182<br/>""]","[""G02B26/10 (EP,US);"", ""G02B26/101 (EP,US);"", ...",[]
3,EP0885233,TERPYRIDINE-PLATINUM(II) COMPLEXES,No opposition filed within time limit,25.04.2003,No opposition filed within time limit,"{""EP01121776.7"": ""EP1164138""}",{},[],[],[GB19960001603],[],"[""C07F15/00, C07D213/06, C07D213/22, C07H19/06...","[""C07H21/00 (EP,US);"", ""A61P33/00 (EP);"", ""A61...",[]
4,EP1105513,METHODS AND COMPOSITIONS FOR USE IN SPLICEOSOM...,The application is deemed to be withdrawn,14.01.2005,Application deemed to be withdrawn,"{""EP04077408.5"": ""EP1489185""}",{},[],[],"[US19980133717, US19980158863]",[],"[""C12N15/90, A61K48/00, C12N15/11, // A61P11:0...","[""C12N15/1027 (EP,US);"", ""A61K48/00 (EP,US);"",...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,EP3937083,SMART METAL CARD WITH RADIO FREQUENCY (RF) TRA...,Examination is in progress,15.11.2023,New entry: Renewal fee paid,{},"{""EP15874266.8"": ""EP3238139""}",[],[],[],[],"[""G06K19/077, G06K19/06<br/>""]","[""G06K19/07773 (EP,US);"", ""G06K19/02 (EP,US);""...",[]
144,EP3992203,PLANT CYTOCHROME P450,Request for examination was made,29.07.2024,New entry: Renewal fee paid,{},{},[],[],"[GB20100021707, GB20100012262]",[],"[""C07K14/415, C12N15/82<br/>""]","[""C07K14/415 (EP,US);"", ""C12N15/8251 (US);"", ""...",[]
145,EP4036079,COMPOUNDS AND COMPOSITIONS FOR INTRACELLULAR D...,Examination is in progress,09.07.2024,New entry: Reply to examination report,{},"{""EP16831870.7"": ""EP3394030""}",[],[],[],[],"[""C07D211/14, C07D211/16, C07D295/13, C07D295/...","[""C07D211/14 (EP,US);"", ""A61K9/5123 (US);"", ""A...",[]
146,EP4215191,COMPOSITION COMPRISING A FUMARATE FOR USE IN A...,Request for examination was made,02.02.2024,The date on which the examining division becom...,{},{},[],[],[],[],"[""A61K31/225, A61P25/02<br/>""]","[""A61K31/225 (EP,CN,IL,KR,US);"", ""A61K47/542 (...",[]


In [12]:
def extraer_valoresV1(texto):
    # Extraer número de patente
    patent = re.findall(r'<span class="highlight">EP<span class="highlight">(\d+)</span></span>', texto)
    if len(patent) == 0:
        patent = ''
    else:
        patent = 'EP' + patent[0]

    # Extraer nombre de la patente
    patent_name = re.findall(r'</span></span> - (.*?)</a>', texto)
    if len(patent_name) == 0:
        patent_name = ''
    else:
        patent_name = patent_name[0].strip()  

    # Extraer Status
    status = re.findall("""Status</td><td class="t2" colspan="3">.*?<br/>""", texto)
    if len(status) == 0:
        status = ''
    else:
        status = status[0]
        status = status.replace("""Status</td><td class="t2" colspan="3">""","").replace('<br/>','').strip()

    #Most Recent Event date
    most_recent_event_date = re.findall(r'Most recent event.*?<td class="t2">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event_date) == 0:
        most_recent_event_date = ''
    else:
        most_recent_event_date = most_recent_event_date[0].strip()

    #Most recent event
    most_recent_event = re.findall(r'Most recent event.*?<td class="t3">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event) == 0:
        most_recent_event = ''
    else:
        most_recent_event = most_recent_event[0].replace('<br/>', '').replace('\r\n', '').replace('\t', '').replace('\xa0', ' ').strip()

    #Divisional
    divisional_pattern = r'<td class="th"[^>]*>Divisional application\(s\)<\/td>(.*?)<\/tr>'
    divisional_match = re.search(divisional_pattern, texto, re.DOTALL)
    divisional_dict = {}
    if divisional_match:
        divisional_text = divisional_match.group(1)
        divisional_apps = re.findall(r'(EP\d+\.\d+)\s*\/\s*<a.*?>(EP\d+)<\/a>', divisional_text)
        divisional_dict = dict(divisional_apps)

    #Parent
    parent_pattern = r'<td class="th"[^>]*>Parent application\(s\).*?<\/td>(.*?)<\/tr>(?:<tr>)?(?:<td[^>]*>)?(.*?)<\/tr>'
    parent_match = re.search(parent_pattern, texto, re.DOTALL | re.IGNORECASE)
    parent_dict = {}
    if parent_match:
        parent_text = parent_match.group(1) + parent_match.group(2)
        parent_apps = re.findall(r'(EP\d+\.\d+)\s*&nbsp;\/\s*<a.*?>(EP\d+)<\/a>', parent_text)
        parent_dict = dict(parent_apps)

    # Extraer inventores
    inventors = re.findall(r'<td class="t2" colspan="3">(\d+)\xa0/\r\n\t(.*?)<br/>\r\n\t(.*?)<br/>\r\n\t(.*?)\r\n\t  / (.*?)<br/>', texto)
    inventors_dict = []
    for inv in inventors:
        inventor_info = {
            'number': inv[0],
            'name': inv[1],
            'address': inv[2] + ', ' + inv[3],
            'country': inv[4]
        }
        inventors_dict.append(inventor_info)

    # Extraer publicaciones 
    publications = re.findall(r'<td class="th">Type:\r\n\s*</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">No.:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Date:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Language:</td>\s*<td class="t2" colspan="2">(.*?)</td>', texto)
    publications_list = []
    for pub in publications:
        # Extraer el texto del enlace si existe, o usar el texto plano si no hay enlace
        type_text = re.search(r'>([^<]+)</a>$', pub[0])
        if type_text:
            pub_type = type_text.group(1).strip()
        else:
            pub_type = re.sub(r'\xa0', ' ', pub[0]).strip()
        publication_info = {
            'type': pub_type,
            'number': re.sub(r'<.*?>', '', pub[1]).strip(),
            'date': pub[2].strip(),
            'language': pub[3].strip()
        }
        publications_list.append(publication_info)
    
    # Crear DataFrame
    df = pd.DataFrame({
        'Patent': [patent],
        'Patent_Name': [patent_name],
        'Status': [status],
        'Most_Recent_Event_Date': [most_recent_event_date],
        'Most_Recent_Event': [most_recent_event],
        'Divisional_applications': [json.dumps(divisional_dict)],
        'Parent_applications': [json.dumps(parent_dict)],
        'Inventors': [json.dumps(inventors_dict)],
        'Publications': [json.dumps(publications_list)]
    })
    
    return df



In [13]:
carpeta = 'C:/Users/iponc/Downloads/HTMLS'  # Ruta de la carpeta HTMLS
df_general = pd.DataFrame()  

for nombre_archivo in os.listdir(carpeta):
    try:
        ruta_completa = os.path.join(carpeta, nombre_archivo)
        if os.path.isfile(ruta_completa):
            print(f"Procesando archivo: {nombre_archivo}")
            
            with open(ruta_completa, 'r', encoding='utf-8') as file:
                contenido = file.read()
                soup = BeautifulSoup(contenido, 'html.parser')
                mytable = extraer_tabla(soup)[0]
                df_actual = extraer_valoresV1(str(mytable))
                
                df_general = pd.concat([df_general, df_actual], ignore_index=True)
                
    except Exception as e:
        print(f"Error inesperado al procesar el archivo {nombre_archivo}: {str(e)}")
        continue

print(f"Total de filas en el DataFrame general: {len(df_general)}")
df_general

Procesando archivo: EP0721594.txt
Procesando archivo: EP0737549.txt
Procesando archivo: EP0788617.txt
Procesando archivo: EP0885233.txt
Procesando archivo: EP1105513.txt
Procesando archivo: EP1127791.txt
Procesando archivo: EP1164138.txt
Procesando archivo: EP1167108.txt
Procesando archivo: EP1168032.txt
Procesando archivo: EP1250728.txt
Procesando archivo: EP1250983.txt
Procesando archivo: EP1250989.txt
Procesando archivo: EP1251150.txt
Procesando archivo: EP1443598.txt
Procesando archivo: EP1475410.txt
Procesando archivo: EP1489185.txt
Procesando archivo: EP1502461.txt
Procesando archivo: EP1613746.txt
Procesando archivo: EP1613796.txt
Procesando archivo: EP1614628.txt
Procesando archivo: EP1616678.txt
Procesando archivo: EP1663881.txt
Procesando archivo: EP1717123.txt
Procesando archivo: EP1730131.txt
Procesando archivo: EP1744959.txt
Procesando archivo: EP1761553.txt
Procesando archivo: EP1769893.txt
Procesando archivo: EP1775298.txt
Procesando archivo: EP1873124.txt
Procesando arc

,Patent,Patent_Name,Status,Most_Recent_Event_Date,Most_Recent_Event,Divisional_applications,Parent_applications,Inventors,Publications
0,EP0721594,AUTONOMOUS CRUISE CONTROL,No opposition filed within time limit,28.12.2007,Lapse of the patent in a contracting state\nNe...,"{""EP01109826.6"": ""EP1167108""}",{},[],[]
1,EP0737549,Abrasive tape and process for producing it,No opposition filed within time limit,24.10.2003,No opposition filed within time limit,"{""EP02012459.0"": ""EP1250983""}",{},[],[]
2,EP0788617,MINIATURE OPTICAL SCANNER FOR A TWO AXIS SCANN...,No opposition filed within time limit,04.01.2008,Lapse of the patent in a contracting state\nNe...,"{""EP01202983.1"": ""EP1168032""}",{},[],[]
3,EP0885233,TERPYRIDINE-PLATINUM(II) COMPLEXES,No opposition filed within time limit,25.04.2003,No opposition filed within time limit,"{""EP01121776.7"": ""EP1164138""}",{},[],[]
4,EP1105513,METHODS AND COMPOSITIONS FOR USE IN SPLICEOSOM...,The application is deemed to be withdrawn,14.01.2005,Application deemed to be withdrawn,"{""EP04077408.5"": ""EP1489185""}",{},[],[]
...,...,...,...,...,...,...,...,...,...
143,EP3937083,SMART METAL CARD WITH RADIO FREQUENCY (RF) TRA...,Examination is in progress,15.11.2023,New entry: Renewal fee paid,{},{},[],[]
144,EP3992203,PLANT CYTOCHROME P450,Request for examination was made,29.07.2024,New entry: Renewal fee paid,{},{},[],[]
145,EP4036079,COMPOUNDS AND COMPOSITIONS FOR INTRACELLULAR D...,Examination is in progress,09.07.2024,New entry: Reply to examination report,{},{},[],[]
146,EP4215191,COMPOSITION COMPRISING A FUMARATE FOR USE IN A...,Request for examination was made,02.02.2024,The date on which the examining division becom...,{},{},[],[]


In [ ]:
df_general.to_csv('C:/Users/iponc/OneDrive/Escritorio/Patents_Filled.csv', index=False)